In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class HousePricePredictor:
    def __init__(self):
        self.model = LinearRegression()
        self.scaler = StandardScaler()

    def load_and_preprocess_data(self, path):
        # Load dataset
        data = pd.read_csv(path)

        # Feature selection
        features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
        X = data[features]
        y = data['SalePrice']

        # Handle missing values
        X = X.fillna(X.mean())

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Feature scaling
        self.scaler.fit(X_train)
        X_train = self.scaler.transform(X_train)

        # Train the model
        self.model.fit(X_train, y_train)

    def predict(self, features):
        # Transform the features using the same scaler
        features = self.scaler.transform([features])
        return self.model.predict(features)[0]

# Instantiate the model and load the data
predictor = HousePricePredictor()
predictor.load_and_preprocess_data('house_prices.csv')  # Replace with the correct path to your dataset


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Apply nest_asyncio
nest_asyncio.apply()

# Define the HousePricePredictor class
class HousePricePredictor:
    def __init__(self):
        self.model = LinearRegression()
        self.scaler = StandardScaler()

    def load_and_preprocess_data(self, path):
        data = pd.read_csv(path)
        features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
        X = data[features]
        y = data['SalePrice']
        X = X.fillna(X.mean())
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        self.scaler.fit(X_train)
        X_train = self.scaler.transform(X_train)
        self.model.fit(X_train, y_train)

    def predict(self, features):
        features = self.scaler.transform([features])
        return self.model.predict(features)[0]

# Instantiate and prepare the model
predictor = HousePricePredictor()
predictor.load_and_preprocess_data('house_prices.csv')  # Replace with your dataset path

# Create FastAPI instance
app = FastAPI()

# Define the Pydantic model for input validation
class HouseFeatures(BaseModel):
    OverallQual: int
    GrLivArea: float
    GarageCars: int
    TotalBsmtSF: float
    FullBath: int
    YearBuilt: int

@app.post("/predict")
def predict_price(features: HouseFeatures):
    try:
        feature_list = [
            features.OverallQual,
            features.GrLivArea,
            features.GarageCars,
            features.TotalBsmtSF,
            features.FullBath,
            features.YearBuilt
        ]
        predicted_price = predictor.predict(feature_list)
        return {"predicted_price": predicted_price}
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

# Run the FastAPI application
uvicorn.run(app, host="127.0.0.1", port=8010)


INFO:     Started server process [13836]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8010 (Press CTRL+C to quit)


INFO:     127.0.0.1:52733 - "POST /predict HTTP/1.1" 200 OK


c:\Users\kaush\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
